#### 1. Business Understanding

JCPenney is a North American department store that was founded in the year 1902 at Kemmerer Wyoming by James Cash Penney. It's headquaters is at Plano in Texas where it is registered as J.C. Penney Corporation, Inc. to trade in goods and services. In goods, JCPenney is a merchant of clothing, foot wears for men, children, women (including plus size), home products such as beddings, bath, toiletories, kitchen wares and windows accessories. Other accessories in the collection of their products include handbags, jewelries and beuaty products. In addition, they are retailers of branded product such as Nike, Levi, Stanford men's tailored clothing etc. They offer services that include styling salon, optical centers, custom decorating etc. To reach its numerous customer base, the organisation has approximately 647 physical stores outlets in America and Puetorico and maintains an appreciable online presence using its e-commerce platform (jcpenney.com, not currently available in the UK) through which it is able to market goods and services. On the average, it is reported that it reached an of approximate of 26 million online view and a staggering 65 million per month during peak holiday season. Owing to this huge online presence and face to face physical contact with customers and clienteles especially during shopping and provision of services such as salon styling, the organisation is therefore required to maintain a business success strategy that entails  customer satisfaction through excellent services, feedback with a view to mitigate intense negative feedback and churn risk. In addition, to maintain a healthy business to cutomer interaction, it is important that the database system void of data corruption to ensure that accurate customer data are used for personalised customer to business interaction, reliable, secure and accurate decision making.
Sequel to the above business understanding, the <b>objective</b> of this research is defined as follows:

1. To identify the spread of the intensity of customer sentiment across United State and Territory
2. To identify weeknesses in the collection and storage of data with a view to mititgate ineffective customer communication and combat the risk of cyber security if neccessary.

**Data to Use:**
1. all columns of reviews.csv data
2. Price and Av_Score columns from products.csv data
3. all columns from jcpenney_reviewers.json
4. all columns of jcpenney_reviewers.json
5. users.csv excluded due to similarities with jcpenney_reviewers.json

#### Abstract : How I answered the above questions to create the project

The strategies implemented is to go from data to actionable business solutions so as to align data initiatives with core business objectives
1. identified customers birthday - no increase in the numbers of birthdays celebrated over the period under review
2. identify duplicated user names
3. identified duplicated empty data structures including strings and list in the databases
4. vader model to gerate polarity scores
5. use kmeans algorithm to fine tune model because the dataset is unlabled
6. use hugging face to generate negative and or positive sentiments
7. The use of linear regression was considered as means to establish relationship between average customer score and price of the product. The process was abandoned because there was no dependable correlation between the columns
8. Comparative analysis was made between the alogortihms used and result dataframe was created from the model and the outcome was used to visualize the sentiments of customers across the united states

#### b. Importing libraries and loading data

In [1]:
##### import libraries for numerical calculations, data and datetime processing
import numpy as np
import pandas as pd
import datetime as dt

# imported for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium

#from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import branca.element as be
import branca.colormap as cm

# for processing text data, build model that generates vader score for intensity of customer ratings
import re 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer # vader score processing
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm.notebook import tqdm

# kmeans is used to cross validate vader output and fine tune the vader score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# to build huggy face ai model and make comparison with other librariew
from transformers import pipeline

import warnings
warnings.filterwarnings("ignore")

In [2]:
#### Load Dataframe
csv_reviews_data = pd.read_csv('data/reviews.csv')
csv_products_data = pd.read_csv('data/products.csv')
csv_user_data = pd.read_csv('data/users.csv')

# The 'lines=True' parameter is used here for json's file to mitigate ValueError: Trailing data
json_reviewers_data = pd.read_json('data/jcpenney_reviewers.json', lines = True)
json_products_data = pd.read_json('data/jcpenney_products.json', lines=True)

In [3]:
states_lat_long_df=pd.read_csv('data/states_data.csv')#Added: Latitude and Longitude data from another source

#### 2. Data understanding and preparation - 
Explore the data and show you understand its structure and relations, with the aid of appropriate visualisation techniques. Assess the data quality, which insights you would be able to answer from it, and what preparation the data would require. Add new data from another source if required to bring new insights to the data you already have.

### Handling unclean data
**(a). null and empty strings**

In [4]:
csv_reviews_data.columns
csv_products_data.columns
json_products_data.columns
json_reviewers_data.columns

Index(['Uniq_id', 'Username', 'Score', 'Review'], dtype='object')

In [12]:
print(f'Data type of list price column: {json_products_data['list_price'].dtypes}')# expected data type = float

Data type of list price column: object


In [13]:
try:
    json_products_data['list_price'].astype(float)
except ValueError as e:
    print(e)

could not convert string to float: ''


In [15]:
def count_empty_string(data):
    """
    Helper function to count numbers of rows that contain empty strings in a column
    """
    count = 0
    for i in data:
        if i == '':
            count += 1
    return count

print(count_empty_string(json_products_data['list_price']))

2166


In [24]:
# Focus is on the price column because SKU and Description are not relevant to this research
csv_products_data.isna().sum()

Uniq_id           0
SKU              67
Name              0
Description     543
Price          2166
Av_Score          0
dtype: int64

observation:
Numbers of empty string in json_products_data['list_price'] = 2166 while numbers of null values in csv_products_data = 2166 (equal amount). 

In [25]:
# They also have equal numbers of rows.
csv_products_data.shape

(7982, 6)

In [26]:
 json_products_data.shape

(7982, 15)

In [28]:
# To ascertain if the index number where json_products_data contains empty data is the same as where csv_products_data contains null data
# Returns True if all of the 2166 rows has matching index numbers
np.unique(json_products_data[json_products_data['list_price']=='']['list_price'].index == csv_products_data[csv_products_data['Price'].isna()].index)

array([ True])

In [30]:
def handle_unclean(data_1, data_2):###############################################################################################################
    """The function is designed to fill null values and empty strings in two pandas series concurrently using the mean of each series. It first 
    converted the datatypes of the series into an object datatypes and add each numeric row to a corresponding list. The mean of the column is obtained 
    by dividing the sum of each list by the length of the corresponding series. The program then sets the new value of the current location where the 
    data is an empty string or a null value with the mean of the corresponding series
    """
    list_data_1 = []
    list_data_2 = []
    if data_1.dtypes != data_1.astype('O'): 
        data_1 = data_1.astype(str)      # converts the values of the row into a string so as to use string method e.g i.isnumeric() below
    if data_2.dtypes != data_2.astype('O'):
        data_2 = data_2.astype(str)
    for i,j in zip(data_1,data_2):
        if i.isnumeric():
            list_data_1.append(float(i))
        if j.isnumeric():
            list_data_2.append(float(j))
    for i in range(len(data_1)):
        if (data_1[i]=='') or (data_2[i]==''):
            data_1[i]=str(np.sum(pd.to_numeric(list_data_1, errors='coerce'))/len(data_1))# assign mean value to every row that contain empty string
            data_2[i]=str(np.sum(pd.to_numeric(list_data_2, errors='coerce'))/len(data_2))
        elif(data_1[i]==np.nan) or (data_2[i]==np.nan):
            data_1[i]=str(np.sum(pd.to_numeric(list_data_1, errors='coerce'))/len(data_1))# assign mean value to every row that contain null value
            data_2[i]=str(np.sum(pd.to_numeric(list_data_2, errors='coerce'))/len(data_2))
    return [data_1, data_2]    
json_products_data['list_price'],csv_products_data['Price'] = handle_unclean(json_products_data['list_price'], csv_products_data['Price'])

In [46]:
print(f'Numbers of empty strings: {json_products_data[json_products_data['list_price']=='']['list_price'].shape[0]}') # to ascertain if the empty 
                                                                                                                      #strings has been removed

Numbers of empty strings: 0


In [44]:
print(f'Numbers of null value: {csv_products_data['Price'].isna().sum()}') # to ascertain if the null values has been removed

Numbers of null value: 0


In [117]:
json_products_data['list_price'].astype(float)

0        41.09
1        41.09
2        41.09
3        41.09
4        41.09
         ...  
7977    201.97
7978    201.97
7979    201.97
7980      0.00
7981      0.00
Name: list_price, Length: 7982, dtype: float64

In [118]:
csv_products_data['Price'].astype(float)

0        41.09
1        41.09
2        41.09
3        41.09
4        41.09
         ...  
7977    201.97
7978    201.97
7979    201.97
7980      0.00
7981      0.00
Name: Price, Length: 7982, dtype: float64

**(b). Empty lists in column rows**

In [56]:
json_reviewers_data.isna().sum()

Username    0
DOB         0
State       0
Reviewed    0
dtype: int64

In [60]:
json_reviewers_data['Reviewed'][0:4]

0                   [cea76118f6a9110a893de2b7654319c0]
1                   [fa04fe6c0dd5189f54fe600838da43d3]
2                                                   []
3    [f129b1803f447c2b1ce43508fb822810, 3b0c9bc0be6...
Name: Reviewed, dtype: object

In [64]:
# complexity of the Reviewed column
print(f'Reviewed column:\nType :{type(json_reviewers_data['Reviewed'])}\nData Type: {json_reviewers_data['Reviewed'].dtypes}\nRow content:{type(json_reviewers_data['Reviewed'][0])}')

Reviewed column:
Type :<class 'pandas.core.series.Series'>
Data Type: object
Row content:<class 'list'>


In [68]:
rows_with_empty_list=json_reviewers_data[json_reviewers_data['Reviewed'].apply(len) == 0] # we target rows that has a len of 0
print(f'Numbers of rows with empty list: {rows_with_empty_list.shape[0]}')

Numbers of rows with empty list: 971


In [69]:
print(f'Alternative code to find numbers of duplicated rows: {json_reviewers_data['Reviewed'].duplicated(keep=False).sum()}')

Alternative code to find numbers of duplicated rows: 971


In [71]:
percentage_rows_with_empty_list=(rows_with_empty_list.shape[0]/json_reviewers_data.shape[0])*100
print(f'Percentage of rows with empty list: {percentage_rows_with_empty_list}%')

Percentage of rows with empty list: 19.42%


**(c). Duplicated Data:**

In [72]:
print(f' Numbers of Username duplicated rows: {json_reviewers_data.Username.duplicated(keep=False).sum()}')# This is in contrast to the result when the 
                                                                                                        # duplicated method was called on the complete dataframe

 Numbers of Username duplicated rows: 2


In [73]:
duplicated_username = json_reviewers_data[json_reviewers_data['Username'].duplicated(keep=False)] # Extract dataframe for duplicated username
duplicated_username

,Username,DOB,State,Reviewed
731,dqft3311,28.07.1995,Tennessee,[5f280fb338485cfc30678998a42f0a55]
2619,dqft3311,03.08.1969,New Mexico,[571b86d307f94e9e8d7919b551c6bb52]


In [79]:
#passing the duplicated username from json_reviewers_data to csv_reviews_data to extract username generated by the username
csv_duplicated_in_username = csv_reviews_data[csv_reviews_data['Username']=='dqft3311']
print(f'Volume of data generated by duplicated username in reviews.csv data:\n{csv_duplicated_in_username.shape[0]} Rows\n{csv_duplicated_in_username[:3]}')

Volume of data generated by duplicated username in reviews.csv data:
17 Rows
                               Uniq_id  Username  Score  \
4331  e5bdf53f2374569526c9f4d55afdd88e  dqft3311      0   
6991  5f280fb338485cfc30678998a42f0a55  dqft3311      2   
8142  6c3c7832675727669a52f794cdec743d  dqft3311      0   

                                                 Review  
4331  We wanted something to warm us from below not ...  
6991  This dress is very pretty and flattering to al...  
8142  The shoes run big and fit very loose. I wear a...  


### Engineering Features

In [91]:
# Creating new dataframe from result obtained from handling unclean data
products_df=pd.merge(csv_products_data[['Uniq_id','Av_Score']].reset_index(),
         json_products_data[['list_price','sale_price','average_product_rating','total_number_reviews']].reset_index()).drop(columns='index')
products_df.head()

,Uniq_id,Av_Score,list_price,sale_price,average_product_rating,total_number_reviews
0,b6c0b6bea69c722939585baeac73c13d,2.625,41.09,24.16,2.625,8
1,93e5272c51d8cce02597e3ce67b7ad0a,3.000,41.09,24.16,3.000,8
2,013e320f2f2ec0cf5b3ff5418d688528,2.625,41.09,24.16,2.625,8
3,505e6633d81f2cb7400c0cfa0394c427,3.500,41.09,24.16,3.500,8
4,d969a8542122e1331e304b09f81a83f6,3.125,41.09,24.16,3.125,8


In [ ]:
# considering that the duplicated  username has a negative multiplier effect, we drop it from both the json_reviewers_data and csv_reviews_data

In [93]:
# Delete username == dqft3311 from jcpenneys_reviewers.json data and save dataframe as reviewers dataframe
reviewers_df = json_reviewers_data.drop(json_reviewers_data[json_reviewers_data['Username'] == 'dqft3311'].index)
# delete Reviewed column from dataframe
reviewers_df.drop(columns=['Reviewed'], inplace=True)

# Delete username == dqft3311 from reviews.csv data and save dataframe as reviews dataframe
reviews_df=csv_reviews_data.drop(csv_reviews_data[csv_reviews_data['Username'] == 'dqft3311'].index)

Detailed research into the dataset indicates that the reveiwers_df is similar to csv_user_data. A quick example is shown by the following two line of codes. As a result, we exclude the csv_use_data to avoid repetition of data.

In [110]:
reviewers_df.head(3)

,Username,DOB,State
0,bkpn1412,31.07.1983,Oregon
1,gqjs4414,27.07.1998,Massachusetts
2,eehe1434,08.08.1950,Idaho


In [111]:
csv_user_data.iloc[0:3,]

,Username,DOB,State
0,bkpn1412,31.07.1983,Oregon
1,gqjs4414,27.07.1998,Massachusetts
2,eehe1434,08.08.1950,Idaho


In [119]:
# To create a state column in the review_df by matching the usernames in the reviewers_df column
# This is a slow method. The map() function provides a faster approach but the following lines of code demonstrate the use of the zip() function
# map() will be used to map latitude and longitudes to the state columns
state_review_list = []
for u in reviews_df['Username']:
    for x,y in zip(reviewers_df['Username'],reviewers_df['State']):
        if u == x:      #if the usernames in both coloumns are the same ..
            state_review_list.append(y)
reviews_df['State']=state_review_list 
reviews_df.isna().sum()

Uniq_id     0
Username    0
Score       0
Review      0
State       0
dtype: int64

**(i). Uniq_id column**

In [149]:
# visual appraisal of the length of Uniq_id column and repeated method of storage
csv_reviews_data['Uniq_id'][:7]

0    b6c0b6bea69c722939585baeac73c13d
1    b6c0b6bea69c722939585baeac73c13d
2    b6c0b6bea69c722939585baeac73c13d
3    b6c0b6bea69c722939585baeac73c13d
4    b6c0b6bea69c722939585baeac73c13d
5    b6c0b6bea69c722939585baeac73c13d
6    b6c0b6bea69c722939585baeac73c13d
Name: Uniq_id, dtype: object

In [140]:
len(csv_reviews_data['Uniq_id'][0]) # to show the number of characters in each row of Uniq_id

32

**(d). observation:**

As revealed above, the dataframe.duplicated(keep=False) method returned a clean bill of health to json_reviewers_data dataframe but when called on Reviewed as a standalone column, it returned 971 duplicated rows. A closer investigation revealed that column is a pandas series with rows that contain lists of string elements. And because an empty list is a hashable value, dataframe.isna().sum() cannot classify it as a null value hence the result. Although this does not expalain why the dataframe.duplicated(keep=False) returned True when there are obvious duplicated rows of empty lists in the Reviewed column, the argument values contained in other rows has the tendency to make each of the rows in the dataframe unique thereby preventing function() to return False on the dataframe.
According to the construct of json_reviewers_data, each Username is allowed to make multiple reviews hence, the Reviewed column is designed to contain a list of as many unique_ids of items that are reviewed by the user. However, because each Username represent a distinct living individual, it becomes questionable when duplicated Username exists in a database. Although from our findings above, only one Username(dqft3311) is found to be duplicated with two entries at index 731 and 2619 which show different DOBs originating from Tenessee and New Mexico. The multiplier effect of this is that when the Username is passed on to csv_reviews_data, it revealed 17 different entries by the same Username but at this time it is not possible to tell if the entry was made by the user from Tennessee or New Mexico. Such duplication has the tendency to lead to flawed Analytics, defective data integrity, operational inefficiency, faulty decision-making and a precusor to cyber vulnerablity.
The length of the Uniq_id column and the repeated mode of storage is a source of conern. 32 characters of Uniq_id that serves the purpose of only identifying a unique product appears to be too long and requires significantly more storage space in the primary table, every index and related tables where the ID is used as a foreign key thereby leading to increased cost of overhead due to storage requirements, human readability and debugginh requirements.

While long unique IDs, such as Universally Unique Identifier(UUIDs), offer advantages in distributed systems for global uniqueness without centralized coordination, jcpenney is currently recognised as an american store hence the use of such long Unique_id come at a cost of local database performance, storage efficiency, increased overhead, more difficult technical requirements to debug and human usability. 

**(e). Data to delete:**

(i). <b><i/>The Reviewed column of the jcpenney_reviewers.json data.</i><b/> 

**Justification to delete:**

   * a. It contains 971 rows of empty list which is a corrupted data
   * b. Deleting the column instead of the rows with empty list will preserve all entries in Username column  which is most relevant to this project
   * c. The information contained in the Reviewed column is still available in the reviews.csv in a different format. 

(ii).<b><i/> Contents of the Username dqft3311</i><b/>

 **Justification to delete:**
* The username is duplicated. Deleting it will ensure data integrity, improve performance of the model, enhance storage capacity and help to mitigate threat of cyber security in the database as a whole

In [151]:
reviews_df.shape

(39046, 4)

**(ii). jcpenney_products.json and products.csv data**

In [154]:
json_products_data.head(1)

,uniq_id,sku,name_title,description,list_price,sale_price,category,category_tree,average_product_rating,product_url,product_image_urls,brand,total_number_reviews,Reviews,Bought With
0,b6c0b6bea69c722939585baeac73c13d,pp5006380337,Alfred Dunner® Essential Pull On Capri Pant,You'll return to our Alfred Dunner pull-on cap...,41.09,24.16,alfred dunner,jcpenney|women|alfred dunner,2.625,http://www.jcpenney.com/alfred-dunner-essentia...,http://s7d9.scene7.com/is/image/JCPenney/DP122...,Alfred Dunner,8,"[{'User': 'fsdv4141', 'Review': 'You never hav...","[898e42fe937a33e8ce5e900ca7a4d924, 8c02c262567..."


In [155]:
json_products_data.columns

Index(['uniq_id', 'sku', 'name_title', 'description', 'list_price',
       'sale_price', 'category', 'category_tree', 'average_product_rating',
       'product_url', 'product_image_urls', 'brand', 'total_number_reviews',
       'Reviews', 'Bought With'],
      dtype='object')

In [167]:
json_products_data.dtypes # to ascertain if the data types of the columns especially list_price and sale_price column

uniq_id                    object
sku                        object
name_title                 object
description                object
list_price                 object
sale_price                 object
category                   object
category_tree              object
average_product_rating    float64
product_url                object
product_image_urls         object
brand                      object
total_number_reviews        int64
Reviews                    object
Bought With                object
dtype: object

In [163]:
# to have a visual appraisal of the multiplier effect of long uniq_id
json_products_data[['uniq_id','Bought With']][0:2]

,uniq_id,Bought With
0,b6c0b6bea69c722939585baeac73c13d,"[898e42fe937a33e8ce5e900ca7a4d924, 8c02c262567..."
1,93e5272c51d8cce02597e3ce67b7ad0a,"[bc9ab3406dcaa84a123b9da862e6367d, 18eb69e8fc2..."


In [165]:
json_products_data['Bought With'][0]

['898e42fe937a33e8ce5e900ca7a4d924',
 '8c02c262567a2267cd207e35637feb1c',
 'b62dd54545cdc1a05d8aaa2d25aed996',
 '0da4c2dcc8cfa0e71200883b00d22b30',
 '90c46b841e2eeece992c57071387899c']

In [168]:
print(f' Total numbers of characters in Unique_id Columns: {json_products_data['uniq_id'].shape[0] * len(json_products_data['uniq_id'][0])}')

 Total numbers of characters in Unique_id Columns: 255424


**(b). Findings: the length of the uniq_id json_products_data**

Findings from the precedding programs revealed that each row of the uniq_id contain 32 character string and the number of rows equals to 7982 as a result of which the total numbers of string characters contained is equal to twenty five million and five thousand and four hundred and twenty four (255424) characters. This suggests that if each row is reduced by 50%, the organisation can save 50% on storage cost, and improve human readability, speed of processing and mitigate the disadvantage of the multiplier effect when the volume of the data increases.

**(c).columns to Delete**

* uniq_id - The length of the column is predicted to slow down data process during nltk data processing and values of the rows has no direct bearing to customer sentiments, howerever it is currently retained for a possible future pd.merge() 
* sku - Is predicted to have similar underlying pattern with unique id and not relevant to customers sentiments
* name_title - Not relevant to customers sentiments
* product_url', 'product_image_urls', 'brand','Bought With' - are not relevant to customers sentiments


**(c).handle unclean data in product['sale_price']**

In [124]:
try:
    products_df['sale_price'].astype(float)
except ValueError as e:
    print(e)

In [123]:
products_df['sale_price']=products_df['sale_price'].str.extract(r'(.{4})', expand=False).astype(float) 
products_df['sale_price'].dtypes

dtype('float64')

In [242]:
print(f'Numbers of remaining null values in products_df: {products_df['sale_price'].isna().sum()}')

Numbers of remaining null values in products_df: 0


In [240]:
# handle null data in product['sale_price']
products_df['sale_price'].fillna(products_df['sale_price'].mean(), inplace=True)
products_df['sale_price'].isna().sum()

In [245]:
products_df.describe() # to review the central tendency, dispersion, and shape of a dataset's distribution,

,total_number_reviews,sale_price,Av_Score
count,7982.000000,7982.000000,7982.000000
mean,4.893886,98.719813,2.988683
std,3.314284,336.696269,0.911673
min,1.000000,3.450000,1.000000
25%,2.000000,22.900000,2.500000
50%,4.000000,35.400000,3.000000
75%,8.000000,60.400000,3.500000
max,23.000000,8720.000000,5.000000


In [246]:
products_df.describe(include='object')

,uniq_id,list_price
count,7982,7982
unique,7982,1037
top,2cc49292b44cc12fe22206440d3e7472,0.0
freq,1,2166


**(d). users.csv**

As shown below, during the research stage, the users.csv and reviewers.jsons file are found to have a lot of similarities. As a result, the users.csv was excluded from the final dataset because the reviewers_df has been created from reviwers.json

In [250]:
csv_user_data.head(2)

,Username,DOB,State
0,bkpn1412,31.07.1983,Oregon
1,gqjs4414,27.07.1998,Massachusetts


In [251]:
reviewers_df.head(2)

,Username,DOB,State
0,bkpn1412,31.07.1983,Oregon
1,gqjs4414,27.07.1998,Massachusetts


To conlcude this section, we have carried out exploratory data analysis on the raw datasets and extracted the following dataframes for use in our final models:
1. reviewers_df
2. products_df
3. reviews_df

####  3. **Data modeling (optional)** - Would modeling be required for the insights you have considered? Use appropriate techniques, if so.
**(i). Linear Regression model to show relationship between scores and price**